In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# file_path = '/Users/forhad/ChromeDriver/chromedriver'
# driver = webdriver.Chrome(file_path)

from selenium.webdriver.chrome.service import Service
ser = Service("/Users/forhad/ChromeDriver/chromedriver")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)

In [3]:
def get_url(address):
    """Generate url from division, district, thana"""
    template = 'https://www.jagonews24.com/bangladesh/{}'
    address = address.replace(' ', '/')
    return template.format(address)

In [4]:
url = get_url('dhaka dhaka')
driver.get(url)

In [5]:
all_buttons = driver.find_elements_by_tag_name("button")


/var/folders/hp/265srph925n8mbmg8mlh2y300000gn/T/ipykernel_4708/3895241270.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  all_buttons = driver.find_elements_by_tag_name("button")


In [6]:
try:
    while all_buttons[2].text == 'আরও পড়ুন':
        time.sleep(5)
        load_more = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="load_more_button"]'))).click()
        if all_buttons[2].text == 'আর কোনো খবর খুজে পাওয়া যায়নি!':
            break
except:
    pass

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div', attrs={'class':'paddingTop10 paddingRight10'})
len(results)

8495

In [8]:
def extract_record(item):
    """Extract and return data into single record"""
    news = item.find('a')['title']
    link = item.find('a')['href']
    result = (news, link)
    return result

In [9]:
records = []
results = soup.find_all('div', attrs={'class':'paddingTop10 paddingRight10'})

for item in results:
    records.append(extract_record(item))

In [10]:
dfM = pd.DataFrame(records, columns=['Headline', 'URL'])
dfM

,Headline,URL
0,১২ বছরের নিচে শিক্ষার্থীদের এখনই ভ্যাকসিন নয়: ...,https://www.jagonews24.com/country/news/731799
1,ঢাবিতে ভর্তির সুযোগ না পেয়ে আত্মহত্যা,https://www.jagonews24.com/country/news/731784
2,জাবিতে সেলিম আল দীনের প্রয়াণ দিবস পালন,https://www.jagonews24.com/country/news/731383
3,বেতনের দাবিতে অনন্ত জলিলের কারখানায় শ্রমিকদের ...,https://www.jagonews24.com/country/news/731222
4,‘মহামারিতে দেশের প্রতি দুইজনের একজন মানসিক সমস...,https://www.jagonews24.com/campus/news/730566
...,...,...
8490,সড়ক দুর্ঘটনা প্রতিরোধ আইন বাতিল ঘোষণা,https://www.jagonews24.com/national/news/7254
8491,উত্তরায় ৩ আফ্রিকান নাগরিক আটক,https://www.jagonews24.com/national/news/7232
8492,পুলিশি শক্তি ছাড়া সরকারের কোনো শক্তি নেই : রিজভী,https://www.jagonews24.com/politics/news/5701
8493,ঢাকা আলিয়া মাদ্রাসায় ছাত্রলীগের দুই গ্রুপের সং...,https://www.jagonews24.com/campus/news/2783


In [12]:
dfM.to_csv('news.csv')